## Семинар 10
# Тема: Предобработка данных: перевод категориальных признаков в числовые

**NOMINAL** - те категории данных, которые не имеют порядка (названия городов) **(ONE-HOT ENCODER/GET DUMMIES)**  <br>
**ORDINAL** - те категории данных, в которых важен порядок (богатый/бедный или Старшая/Средняя) **(LABEL ENCODER)**

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import metrics

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder

https://www.kaggle.com/lodetomasi1995/income-classification

### 1. Загрузите датасет `income-classification.csv` и вывеите первые 5 строк.

In [33]:
df = pd.read_csv('income_evaluation.csv')
df.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### 2. Выведите информацию о загруженном датафрейме и его размеры. 

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              32561 non-null  int64 
 1    workclass       32561 non-null  object
 2    fnlwgt          32561 non-null  int64 
 3    education       32561 non-null  object
 4    education-num   32561 non-null  int64 
 5    marital-status  32561 non-null  object
 6    occupation      32561 non-null  object
 7    relationship    32561 non-null  object
 8    race            32561 non-null  object
 9    sex             32561 non-null  object
 10   capital-gain    32561 non-null  int64 
 11   capital-loss    32561 non-null  int64 
 12   hours-per-week  32561 non-null  int64 
 13   native-country  32561 non-null  object
 14   income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [35]:
df.shape

(32561, 15)

### 3. Убедитесь, что в данных нет пропущенных значений и выведите названия колонок с типом данных object. 

In [36]:
df.isnull().sum()

age                0
 workclass         0
 fnlwgt            0
 education         0
 education-num     0
 marital-status    0
 occupation        0
 relationship      0
 race              0
 sex               0
 capital-gain      0
 capital-loss      0
 hours-per-week    0
 native-country    0
 income            0
dtype: int64

In [37]:
df_object = df.select_dtypes(include=['object'])
df_object.columns

Index([' workclass', ' education', ' marital-status', ' occupation',
       ' relationship', ' race', ' sex', ' native-country', ' income'],
      dtype='object')

### 4. Распределите признаки с типом данных object на номинальные и ординальные. Выведите количество уникальных значений в каждом из них.

In [38]:
df_object[' workclass'].value_counts()

 workclass
Private             22696
Self-emp-not-inc     2541
Local-gov            2093
?                    1836
State-gov            1298
Self-emp-inc         1116
Federal-gov           960
Without-pay            14
Never-worked            7
Name: count, dtype: int64

In [39]:
nominals = [' race', ' sex', ' native-country', ' occupation']
ordinals = [' workclass', ' education', ' marital-status', ' relationship']

In [40]:
df_object[nominals].nunique()

race               5
sex                2
native-country    42
occupation        15
dtype: int64

In [41]:
df_object[ordinals].nunique()

workclass          9
education         16
marital-status     7
relationship       6
dtype: int64

### 5. Отделите целевую переменную и замените в ней значения на числовые при помощи LabelEncoder.

In [42]:
y = df[' income']
X = df.drop(' income', axis= 1)

In [43]:
y.value_counts()

 income
<=50K    24720
>50K      7841
Name: count, dtype: int64

In [44]:
labelencoder = LabelEncoder()
y_encoded = labelencoder.fit_transform(y)
pd.Series(y_encoded).value_counts()

0    24720
1     7841
Name: count, dtype: int64

### 6. Создайте копию датафрейма с признаками и замените в нём все категориальные ординальные признаки, а также признак sex на числовые при помощи OrdinalEncoder.

In [45]:
ordinalencoder = OrdinalEncoder()
ordinals_encoded = ordinalencoder.fit_transform(df_object[ordinals])
ordinals_encoded

array([[ 7.,  9.,  4.,  1.],
       [ 6.,  9.,  2.,  0.],
       [ 4., 11.,  0.,  1.],
       ...,
       [ 4., 11.,  6.,  4.],
       [ 4., 11.,  4.,  3.],
       [ 5., 11.,  2.,  5.]], shape=(32561, 4))

In [46]:
df_ordinasls = pd.DataFrame(ordinals_encoded, columns= ordinals)
df_ordinasls

,workclass,education,marital-status,relationship
0,7.0,9.0,4.0,1.0
1,6.0,9.0,2.0,0.0
2,4.0,11.0,0.0,1.0
3,4.0,1.0,2.0,0.0
4,4.0,9.0,2.0,5.0
...,...,...,...,...
32556,4.0,7.0,2.0,5.0
32557,4.0,11.0,2.0,0.0
32558,4.0,11.0,6.0,4.0
32559,4.0,11.0,4.0,3.0


### 7. Создайте копию датафрейма и замените в ней категориальный номинальный признак race на числовой при помощи OneHotEncoder.

In [47]:
df_object[' race'].value_counts()

 race
White                 27816
Black                  3124
Asian-Pac-Islander     1039
Amer-Indian-Eskimo      311
Other                   271
Name: count, dtype: int64

In [48]:
onehotencoder = OneHotEncoder(drop = 'first', sparse_output=False)
nominals_encoded = onehotencoder.fit_transform(df_object[[' race']])
nominals_encoded

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], shape=(32561, 4))

In [49]:
onehotencoder.get_feature_names_out()

array([' race_ Asian-Pac-Islander', ' race_ Black', ' race_ Other',
       ' race_ White'], dtype=object)

In [52]:
df_nominals = pd.DataFrame(nominals_encoded, columns= onehotencoder.get_feature_names_out()).astype('int64')
df_nominals.head()

,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,1,0,0
4,0,1,0,0


In [53]:
df_ordinasls.join(df_nominals)

,workclass,education,marital-status,relationship,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White
0,7.0,9.0,4.0,1.0,0,0,0,1
1,6.0,9.0,2.0,0.0,0,0,0,1
2,4.0,11.0,0.0,1.0,0,0,0,1
3,4.0,1.0,2.0,0.0,0,1,0,0
4,4.0,9.0,2.0,5.0,0,1,0,0
...,...,...,...,...,...,...,...,...
32556,4.0,7.0,2.0,5.0,0,0,0,1
32557,4.0,11.0,2.0,0.0,0,0,0,1
32558,4.0,11.0,6.0,4.0,0,0,0,1
32559,4.0,11.0,4.0,3.0,0,0,0,1


### 8. Создайте копию датафрейма и замените в ней категориальный номинальный признак race на числовые при помощи get_dummies.

In [55]:
df_dum = df.copy()
pd.get_dummies(df_dum, columns=[' race'], dtype='int64').head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country,income,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,Male,2174,0,40,United-States,<=50K,0,0,0,0,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,13,United-States,<=50K,0,0,0,0,1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,Male,0,0,40,United-States,<=50K,0,0,0,0,1
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Male,0,0,40,United-States,<=50K,0,0,1,0,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Female,0,0,40,Cuba,<=50K,0,0,1,0,0


### Задания для самостоятельного выполнения:

1. Загрузите датасет `House_Rent_Dataset.csv` для предсказании арендной платы в зависимости от ряда признаков. Убедитесь, что в датасете нет пропущенных значений. Посмотрите размер датасета. 

Поработайте с категориальными признаками:

2. Убедитесь в том, что столбец `Area Locality` является столбцом с практически уникальными значениями, удалите его.
3. Сделайте так, чтобы признаки `Area Type` и `Point of Contact` имели только по два значения. Для этого удалите строки с редко встречающимся третьим значением. Замените оставшиеся значения на `0` и `1`. 
4. Определите, сколько значений имеет признак `City`, а сколько `Tenant Preferred`. Сделайте из этих признаков dummy переменные, т.е. замените каждый признак на такое количество признаков, сколько значений он принимает, где в каждом новом признаке на месте его значения стоят единицы, а остальные - нули. После этого по одному из столбцов удалите. Присоедините полученные столбцы к исходному датафрейму, а столбцы `City` и `Tenant Preferred` удалите.
5. Замените признак `Furnishing Status` на числовой при помощи LabelEncoder.
6. В дате `Posted On` оставьте только номер месяца, в котором размещено объявление о сдаче в аренду. Тип данных должен быть - целое число.
7. Превратите признак `Floor` в два признака: `Floor_number`- номер этажа, на котором находится квартира (подвалы обозначьте числом `0`) и `Total_floors`- общее количество этажей в доме, в котором сдаётся квартира. Полученные столбцы переведите в тип int.
8. Сохраните полученный датасет и выведете его.
9. Отделите целевую переменную Rent и разбейте данные на обучающую и тестовую выборки. Обучите модель линейной регрессии на обучающих данных и оцените ее эффективность на тестовом наборе.
